### pytest fixture

In [ ]:
# Kas yra fixture?
# Fixture – tai speciali funkcija, kuri paruošia duomenis arba testų aplinką, kurią testų funkcijos gali naudoti kaip parametrą. Tai leidžia:
#   Vengti kartojimo testuose.
#   Sutvarkyti sudėtingus paruošimo veiksmus.
#   Valdyti testų priklausomybes.

# Kaip sukurti fixture?
# Naudojame dekoratorių @pytest.fixture virš funkcijos.

In [ ]:
# Pavyzdys:
import pytest

@pytest.fixture
def sample_list():
    # paruošiame testui sąrašą
    return [1, 2, 3, 4, 5]

In [ ]:
# fixture naudojimas su klasėmis

import pytest

@pytest.fixture
def pradine_suma():
    return 100

class TestAritmetika:
    def test_pridejimas(self, pradine_suma):
        assert pradine_suma + 50 == 150

    def test_atimtis(self, pradine_suma):
        assert pradine_suma - 20 == 80
# fixture čia perduodama automatiškai kiekvienam testui klasėje.

In [ ]:
# Galimi scope variantai:

@pytest.fixture(scope="function")  # numatytasis
@pytest.fixture(scope="class")
@pytest.fixture(scope="module")
@pytest.fixture(scope="package")
@pytest.fixture(scope="session")

# function – fixture sukuriama ir sunaikinama kiekvienai test funkcijai.
# class – fixture sukuriama vieną kartą kiekvienai testų klasei (naudojama su self).
# module – fixture sukuriama vieną kartą visam failui.
# package – vieną kartą visam __init__.py turinčiam paketui.
# session – vieną kartą visai testų sesijai (naudinga pvz. jungiantis prie DB ar paleidžiant naršyklę).

#### scope="function"

In [ ]:
# scope="function" reiškia, kad fixture bus paleista iš naujo kiekvienai testavimo funkcijai (arba metodui).
# Tai naudinga, kai:
#     Reikia švarios, izoliuotos aplinkos kiekvienam testui.
#     Nenori, kad testai darytų įtaką vieni kitiems.

# Pavyzdys: 
import pytest

@pytest.fixture
def prepare_data():
    print("Setup")
    return {"key": "value"}

def test_one(prepare_data):
    print("Test One")
    assert prepare_data["key"] == "value"

def test_two(prepare_data):
    print("Test Two")
    assert "key" in prepare_data


#### scope="class"

In [ ]:
# Kai pytest fixture aprašoma su scope="class", ji sukuriama vieną kartą
# visai testų klasei, o ne kiekvienam testui atskirai. Tai panašu į setup_class,
# bet lankstesnis būdas.

# Pavyzdys:
import pytest
from skaiciavimai import daugyba

# Fixture su scope="class"
@pytest.fixture(scope="class")
def bendri_duomenys():
    print("🔧 Sukuriami bendri duomenys (vieną kartą klasei)")
    return {"koef": 10}

class TestDaugyba:
    def test_pirmas(self, bendri_duomenys):
        assert daugyba(2, bendri_duomenys["koef"]) == 20

    def test_antras(self, bendri_duomenys):
        assert daugyba(3, bendri_duomenys["koef"]) == 30

# Paaiškinimas
# @pytest.fixture(scope="class") reiškia, kad bendri_duomenys bus sukurtas vieną kartą prieš visus klasės testus.
# Tai naudinga, kai duomenys ar resursai nesikeičia tarp testų.
# Jis iškviečiamas tik vieną kartą, o ne kiekvienam testui.


#### scope="module"

In [ ]:
# Kas yra scope="module"?
# Fixture bus paleista vieną kartą visam testų failui (moduliui).
# Nepriklauso nuo klasių ar funkcijų skaičiaus.
# Visi testai tame pačiame .py faile gaus tą pačią fixture reikšmę.

# Pavyzdys:
import pytest

@pytest.fixture(scope="module")
def shared_data():
    print("⚙️ Fixture vyksta")
    return 42

def test_one(shared_data):
    print("🧪 test_one:", shared_data)
    assert shared_data == 42

def test_two(shared_data):
    print("🧪 test_two:", shared_data)
    assert shared_data + 1 == 43

class TestClass:
    def test_three(self, shared_data):
        print("🧪 test_three:", shared_data)
        assert shared_data * 2 == 84

# Rezultatas:
⚙️ Fixture vyksta
🧪 test_one: 42
🧪 test_two: 42
🧪 test_three: 42
# Paaiskinimas:
# Fixture paleista tik kartą, nes scope="module".
# Visi testai (funkcijos + klasės metodai) tame pačiame .py faile
# naudojasi ta pačia reikšme 42.

#### scope="package"

In [ ]:
# Fixture su scope="package" vykdoma vieną kartą visam paketui.
# Tai reiškia: visiems .py failams viename aplanke (pakete) bus panaudota viena ir ta pati fixture reikšmė.
# Visi testai tame pakete (aplanke su __init__.py) dalijasi tą pačią fixture.

# Paprastai:

# tests/
#   ├── __init__.py       ← Šitas failas padaro aplanką „paketu“
#   ├── conftest.py       ← Čia tavo fixture
#   ├── test_file1.py
#   ├── test_file2.py

# conftest.py
import pytest

@pytest.fixture(scope="package")
def shared_package_data():
    print("⚙️ Fixture vyksta")
    return "Shared across package"

#test_file1.py
def test_one(shared_package_data):
    print("🧪 test_one:", shared_package_data)
    assert shared_package_data == "Shared across package"

#test_file2.py
def test_two(shared_package_data):
    print("🧪 test_two:", shared_package_data)
    assert "Shared" in shared_package_data

#  Konsolėje pamatysi:
⚙️ Fixture vyksta
🧪 test_one: Shared across package
🧪 test_two: Shared across package

# Kada naudoti scope="package"?
#   Kai turi kelis failus tame pačiame aplanke, ir:
#   Tavo fixture duomenys gali būti bendrinami tarp visų failų
#   Norisi vieno pasiruošimo (setup) visam aplankui
# Pvz.: API login sesija, prisijungimas prie testinės duomenų bazės, stambus failų nuskaitymas

#### scope="session"

In [ ]:
# Fixture su scope="session" bus paleista tik vieną kartą visame pytest paleidime – 
#net jei testai išsibarstę per keliolika failų ir aplankų.
# Tai efektyvu, kai reikia:
#   prisijungti prie testinės DB tik vieną kartą,
#   autentifikuotis į API sesiją,
#   įkelti didelį failą į atmintį,
#   sukurti bendrus testinius duomenis.

# Struktūra:
# tests/
#   ├── __init__.py
#   ├── conftest.py        ← Čia mūsų session fixture
#   ├── test_one.py
#   ├── test_two.py

#conftest.py
import pytest

@pytest.fixture(scope="session")
def shared_session_data():
    print("⚙️ [SESSION] Fixture vyksta")
    return "Shared across all tests"

# test_one.py
def test_first(shared_session_data):
    print("🧪 test_first:", shared_session_data)
    assert "Shared" in shared_session_data

# test_two.py
def test_second(shared_session_data):
    print("🧪 test_second:", shared_session_data)
    assert shared_session_data.endswith("tests")

# Paleidus pytest:
⚙️ [SESSION] Fixture vyksta
🧪 test_first: Shared across all tests
🧪 test_second: Shared across all tests

#### Fixture'ų konfigūravimas su params (parametrizavimas)

In [ ]:
# Kas yra params fixture'e?
# Kai nurodai params=[…] fixture'e, pytest automatiškai paleis
# testą kelis kartus, kiekvieną kartą su skirtinga reikšme iš sąrašo.

# Tai panašu į @pytest.mark.parametrize, bet veikia fixture'e.

# Galimi tipai fixture parametruose:
params = [
    "string",
    123,
    (1, 2),
    {"key": "value"},
    True,
    None
]

#Pavyzdys:
import pytest

# Fixture su trim skirtingom reikšmėm
@pytest.fixture(params=[1, 2, 3])
def number(request):
    print(f"⚙️ Fixture vyksta su reikšme: {request.param}")
    return request.param

def test_even(number):
    print(f"🧪 Testas su skaičiumi: {number}")
    assert number in [1, 2, 3]

# Rezultatas:
# ⚙️ Fixture vyksta su reikšme: 1
# 🧪 Testas su skaičiumi: 1
# .
# ⚙️ Fixture vyksta su reikšme: 2
# 🧪 Testas su skaičiumi: 2
# .
# ⚙️ Fixture vyksta su reikšme: 3
# 🧪 Testas su skaičiumi: 3
# .

# Kas įvyks paleidus pytest?
# Fixture bus paleista 3 kartus: su 1, 2 ir 3.
# Testas test_even bus paleistas 3 kartus su tom reikšmėm.

#Prieiga prie reikšmės: request.param
# Kad gautum konkrečią reikšmę, fixture naudoja request objektą:
def number(request):
    return request.param

# Kada naudoti params fixture'e?
#   Kai nori paleisti tą patį testą su skirtingais duomenimis
#   Kai tie duomenys susiję su setup logika
#   Pvz.: testuoti API su skirtingais vartotojais, DB konfigūracijomis, įvairiais failais ir t. t.

In [ ]:
# Pavyzdys:

# Scenarijus: Vartotojų leidimų testavimas su dict ir params
# Tikslas:
# Testuoti, ar vartotojas turi teisę:
# peržiūrėti (view)
# trinti (delete)

 # conftest.py
import pytest

@pytest.fixture(params=[
    {"role": "admin", "can_view": True, "can_delete": True},
    {"role": "editor", "can_view": True, "can_delete": False},
    {"role": "viewer", "can_view": True, "can_delete": False},
    {"role": "guest", "can_view": False, "can_delete": False},
])
def user_data(request):
    return request.param

# test_permissions.py
def test_can_view(user_data):
    print(f"Testing role: {user_data['role']}")
    allowed_roles = ["admin", "editor", "viewer"]
    assert (user_data["role"] in allowed_roles) == user_data["can_view"]

def test_can_delete(user_data):
    print(f"Testing role: {user_data['role']}")
    assert (user_data["role"] == "admin") == user_data["can_delete"]
# Konsolės output bus maždaug toks:
# Testing role: admin
# Testing role: editor
# Testing role: viewer
# Testing role: guest
# Testing role: admin
# Testing role: editor
# Testing role: viewer
# Testing role: guest

# Pytest paleidžia:
# Abi funkcijas po 4 kartus
# Pagal kiekvieną dict'ą iš fixture.
# 8 paleidimai viso.

In [ ]:
# Užduotis: Tikrinti vartotojo teises pagal rolę
# 🎯 Tikslas:
# Parašyti fixture su skirtingomis vartotojo rolėmis ir testus, kurie patikrina, ką vartotojas gali daryti (arba ne).
# ✅ Žingsniai:
# 1️⃣ Sukurk conftest.py failą
# Čia įrašysi fixture su params=["admin", "editor", "guest"].
# Fixture grąžins šią string reikšmę naudodama request.param.
# 2️⃣ Sukurk test_roles.py failą
# Sukurk testą test_can_edit_content(role):
# Tegul leidžiama redaguoti tik "admin" ir "editor".
# "guest" negali.
# 3️⃣ Sukurk testą test_can_delete_content(role)
# Tik "admin" turi leidimą trinti.
# "editor" ir "guest" – neturi.
# 4️⃣ Paleisk pytest ir patikrink, ar testai paleidžiami 3 kartus (kiekvienai rolei).
# 5️⃣ (Pasirinktinai) Į testus įdėk print(...), kad pamatytum, kokia rolė naudojama.

# conftest.py
@pytest.fixture(params=["admin", "editor", "guest"])
def role(request):
  return request.param

# test_role.py
def test_can_edit_content(role):
  print(f"Testing with role: {role}")
  assert role in ["admin", "editor"]

def test_can_delete_content(role):
  print(f"Testing with role: {role}")
  assert role == "admin"

#### yield Python kalboje

In [ ]:
# yield – tai raktinis žodis, kuris leidžia funkcijai tapti generatoriumi.
# Skirtingai nuo return, kuris nutraukia funkciją ir grąžina reikšmę, yield
# sustabdo funkcijos vykdymą ir leidžia vėliau jį tęsti nuo tos vietos.

# Paprastas yield pavyzdys
def count_up_to(n):
    for i in range(1, n + 1):
        yield i

counter = count_up_to(3)

for number in counter:
    print(number)

# Rezultatas:
1
2
3
# yield leidžia sukurti duomenų srautą – naudinga, kai nenori visko laikyti atmintyje.

# yield fixture'e (pytest)
# Kai naudojame yield fixture'e, tai leidžia:
#✅ paleisti setup kodą prieš testą
#✅ grąžinti testui reikalingą reikšmę
#✅ paleisti teardown kodą po testo (net jei testas sugenda)

# Pavyzdys su yield fixture'e:
import pytest

@pytest.fixture
def resource():
    print("⚙️ Setup: Sukuriame resursą")
    yield 42  # ši reikšmė bus perduota į testą
    print("🧹 Teardown: Išvalome resursą")

def test_example(resource):
    print("🧪 Testas vyksta")
    assert resource == 42
# Vykdymo tvarka:
# - Setup vykdomas prieš testą
# - yield 42 perduoda reikšmę į testą
# - Testas vyksta
# - Kai testas baigiasi (net jei sugenda) → 🧹 Teardown vykdomas

#  Kada naudoti yield fixture'e?
# - Kai reikia atidaryti failą / prisijungti prie DB / paleisti serverį
# - Ir tada uždaryti / išvalyti po testų

#  Pvz: fixture su yield ir failu
import pytest

@pytest.fixture
def temp_file(tmp_path):
    file = tmp_path / "test.txt"
    file.write_text("labas")
    yield file
    print("🧹 Failas išvalytas")  # Čia galima trinti failą ar pan.